# Suicide analysis
### Lukas Forst

In [4]:
import pandas as pd
import numpy as np
import plotly as plt

Now let's look at the data.

In [6]:
data = pd.read_csv('data.csv')
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


Now, select random sample from the dataset.

In [8]:
data.sample(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
2227,Azerbaijan,1994,male,5-14 years,0,814500,0.00,Azerbaijan1994,NaN,"3,313,739,674",508,Millenials
11348,Guyana,2012,female,55-74 years,5,38249,13.07,Guyana2012,0.629,"2,861,562,266",4212,Boomers
2030,Austria,2005,male,25-34 years,102,548943,18.58,Austria2005,0.853,"315,974,418,605",40367,Generation X
15215,Lithuania,2009,male,25-34 years,116,205749,56.38,Lithuania2009,NaN,"37,440,673,478",12412,Generation X
11674,Iceland,1985,male,15-24 years,7,21900,31.96,Iceland1985,0.776,"2,939,845,038",13357,Generation X


In [9]:
data.describe()

,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,126352.000000
